<a href="https://colab.research.google.com/github/JayThibs/ai-safety-prize-challenge/blob/main/deploy_ais_prize_challenge_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone project

!git clone https://github.com/unitaryai/detoxify

# create virtual env

!python3 -m venv toxic-env
!source toxic-env/bin/activate

# install project

!pip install -e detoxify
%cd detoxify

# for training
!pip install -r requirements.txt

Cloning into 'detoxify'...
remote: Enumerating objects: 791, done.
remote: Counting objects: 100% (310/310), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 791 (delta 163), reused 196 (delta 83), pack-reused 481
Receiving objects: 100% (791/791), 52.05 MiB | 21.52 MiB/s, done.
Resolving deltas: 100% (440/440), done.
Error: Command '['/content/toxic-env/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: toxic-env/bin/activate: No such file or directory
Obtaining file:///content/detoxify
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 56.8 MB/s 
     |████████████████████████████████| 596 kB 75.4 MB/s 
     |████████████████████████████████| 77 kB 8.9 MB/s 
     |████████████████████████████████| 6.5 MB 58.5 MB/s 
     |████████

In [2]:
!pip install colabcode fastapi transformers==4.17.0 -q

     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 3.8 MB 10.2 MB/s 
     |████████████████████████████████| 745 kB 70.4 MB/s 
     |████████████████████████████████| 45 kB 4.3 MB/s 
     |████████████████████████████████| 8.3 MB 4.5 MB/s 
     |████████████████████████████████| 428 kB 71.3 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 343 kB 90.4 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
     |████████████████████████████████| 130 kB 74.0 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 561 kB 66.7 MB/s 
     |████████████████████████████████| 130 kB 88.8 MB/s 
     |████████████████████████████████| 130 kB 81.5 MB/s 
     |████████████████████████████████| 130 kB 65.6 MB/s 
     |████████████████████████████████| 130 kB 72.5 MB/s 
     |████████████████████

In [3]:
from colabcode import ColabCode
from fastapi import FastAPI
from detoxify import Detoxify
import pandas as pd
import torch
import tensorflow as tf
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
cc = ColabCode(port=12000, code=False)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [6]:
input_text = "I like you. I love you"
input_ids = tokenizer(input_text)
batch = tokenizer(
    input_text,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'
)
output = model(**batch)

In [7]:
from torch import nn
predictions = nn.functional.softmax(output.logits, dim=-1)

In [8]:
predictions

tensor([[0.4845, 0.0812, 0.1261, 0.0977, 0.1116, 0.0988]],
       grad_fn=<SoftmaxBackward0>)

In [15]:
model = Detoxify('original', device='cuda')

In [16]:
results = model.predict(input_text)
print(results)
# print(pd.DataFrame(results,index=input_text))

{'toxicity': 0.00077290746, 'severe_toxicity': 0.00012961477, 'obscene': 0.0002012506, 'threat': 0.00015594847, 'insult': 0.0001781984, 'identity_attack': 0.00015770987}


In [22]:
%%writefile models.py
from pydantic import BaseModel, conlist
from typing import List


class detoxifyModel(BaseModel):
    data: List[conlist(str, min_items=1, max_items=10)]

Overwriting models.py


In [66]:
import logging
from fastapi import FastAPI
from models import detoxifyModel

app = FastAPI(title="AI Safety Prize Challenge Classifier", description="This is a classifier to detect bad outputs from decoder models.", version="1.0")

# Initialize logging
my_logger = logging.getLogger()
my_logger.setLevel(logging.DEBUG)
logging.basicConfig(level=logging.DEBUG, filename='logs.log')

model = None

@app.on_event("startup")
def load_model():
    global model
    model = Detoxify('original', device='cuda')

@app.post("/api", tags=["prediction"])
async def get_predictions(input_list: detoxifyModel):
    try:
        results = {"data": []}
        input_list = dict(input_list)["data"][0]
        for text in input_list:
            predictions = model.predict(text)
            for k, v in predictions.items():
                predictions[k] = float(v)
            results["data"].append(predictions)
        return results
    except:
        my_logger.error("Something went wrong!")
        return ['error']

In [67]:
input_text = {
  "data": [
    [
      "test_string",
     "test_2"
    ]
  ]
}

In [68]:
dict(input_text)["data"][0]

['test_string', 'test_2']

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://3ed0-35-185-109-155.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [81]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     142.162.126.220:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     142.162.126.220:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     142.162.126.220:0 - "GET /openapi.json HTTP/1.1" 200 OK
{'data': []}
['string']
string
{'data': [{'toxicity': 0.01195145957171917, 'severe_toxicity': 0.00010437105811433867, 'obscene': 0.0004463696095626801, 'threat': 0.00014336399908643216, 'insult': 0.0004569115408230573, 'identity_attack': 0.0002834190090652555}]}
INFO:     142.162.126.220:0 - "POST /api HTTP/1.1" 200 OK


In [5]:
app = FastAPI()

@app.get('/')
async def read_root():
    results = model.predict(input_text)
    return results

In [6]:
cc.run_app(app=app)

INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://481d-34-125-115-36.ngrok.io" -> "http://localhost:12000"
INFO:     142.162.126.220:0 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [58]


In [60]:
input = {'toxicity': 0.01195146, 'severe_toxicity': 0.00010437106, 'obscene': 0.0004463696, 'threat': 0.000143364, 'insult': 0.00045691154, 'identity_attack': 0.000283419}

In [61]:
for k, v in input.items():
    input[k] = float(v)
print(type(input['toxicity']))

<class 'float'>
